In [219]:
import matplotlib.pyplot as plt
from copy import deepcopy
import rasterio as rio
import pandas as pd
import numpy as np
import importlib
import sklearn
import random
import pickle
import torch
import math
import copy

import resnet18
importlib.reload(resnet18)
from helpers import *

In [194]:
import warnings
warnings.filterwarnings('ignore')


In [195]:
class dataLoader():

    def __init__(self, dir, df):
        self.data = []
        self.labels = []
        df = pd.read_csv(df)

        for i in os.listdir(dir):
            fname = os.path.join(dir, i)
            im = np.array(rio.open(fname).read(1))
            im = torch.from_numpy(im)
            im = torch.reshape(im, (1, 224, 224)).numpy()
            num_mig = df[df['sending'] == int(fname.split("m")[1].split(".")[0])]['US_MIG_05_10'].to_list()[0]

            self.data.append(im)
            self.labels.append(num_mig)

    def train_val_split(self, split):
        train_num = int(len(self.data) * split)
        train_indices = random.sample(range(0, len(self.data)), train_num)
        val_indices = [i for i in range(0, len(self.data)) if i not in train_indices]
        x_train, y_train = [self.data[i] for i in train_indices], [self.labels[i] for i in train_indices]
        x_val, y_val = [self.data[i] for i in val_indices], [self.labels[i] for i in val_indices]
        return x_train, y_train, x_val, y_val 


In [196]:
d = dataLoader("./final_pics", "./us_migration.csv")

In [197]:
x_train, y_train, x_val, y_val = d.train_val_split(split = .80)

In [220]:
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'

####### Build and fit the Model
lr = 1e-7
batchSize = 100
model = resnet18.resnet18(outDim = batchSize)

In [221]:
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [222]:
def train_model(model, criterion, optimizer, lr, batchSize, num_epochs):

    losses = []
    maes = []

    best_mae = 100000000

    for t in range(num_epochs):

        for phase in ['train', 'val']:

            if phase == 'train':

                #Batches
                batchObs = random.sample(range(0, len(y_train)), batchSize)
                # print(batchObs)
                modelX = [x_train[i] for i in batchObs]
                modelX = torch.tensor(list(modelX), requires_grad = True, dtype = torch.float32)
                modely = torch.tensor([y_train[i] for i in batchObs], dtype = torch.float32)  # MADE A CHANGE HERE 
                y_pred = model(modelX, t)
                
                loss = criterion(y_pred, modely)  
                
                # Zero gradients, perform a backward pass, and update the weights.
                optimizer.zero_grad()
                grad = torch.autograd.grad(outputs=loss, inputs=modelX, retain_graph = True)
                # print("    GRADIENT: ", grad[0][0].shape)
                # print("    GRADIENT: ", grad[0])
                loss.backward()
                optimizer.step()
                # https://discuss.pytorch.org/t/updatation-of-parameters-without-using-optimizer-step/34244/4
                # with torch.no_grad():
                #     for name, p in model.named_parameters():
                #         if name == 'SocialSig.W':
                #         # print("    In with:        ", p.data)
                #             new_val = socialSigLayers.update_function(p, grad[0], loss, lr)
                #         # print("NEW WEIGHTS: ", new_val)
                #             p.copy_(new_val)

                print("EPOCH: ", t)
                # print("    Train")
                epoch_mae = mae(y_pred, modely).item()
                print("    TRAIN    Loss:   ", loss.item(), "     MAE:   ", epoch_mae)
                # print("    Train Preds:     ", y_pred)

            if phase == 'val':

                #Batches
                batchObs = random.sample(range(0, len(y_train)), batchSize)
                # print(batchObs)
                modelX = [x_train[i] for i in batchObs]
                modelX = torch.tensor(list(modelX), requires_grad = True, dtype = torch.float32)
                modely = torch.tensor([y_train[i] for i in batchObs], dtype = torch.float32)  # MADE A CHANGE HERE 
                
                # Perform evaluations of the batch predictions
                y_pred = model(modelX, t)
                
                loss = criterion(y_pred, modely)  
                epoch_mae = mae(y_pred, modely).item()
                print("    VAL      Loss:   ", loss.item(), "     MAE:   ", epoch_mae)
                # print("    Val Preds:       ", y_pred)

                if epoch_mae < best_mae:
                    
                    best_mae = epoch_mae
                    best_model_wts = copy.deepcopy(model.state_dict())
                    model_name = './models/test_Epoch' + str(t) + '.sav'
                    pickle.dump(model, open(model_name, 'wb'))

                losses.append(loss.item())
                maes.append(epoch_mae)
                
        print("\n")

    print("TRAINING COMPLETE")
    print("Best MAE: ", best_mae)

    # print(best_model_wts)

    # return model.load_state_dict(best_model_wts)
    return best_model_wts, losses, maes

In [218]:
num_epochs = 20

best_model_wts, losses, maes = train_model(model, criterion, optimizer, lr, batchSize, num_epochs)

RuntimeError: The size of tensor a (5) must match the size of tensor b (100) at non-singleton dimension 0